# Build an agent for flight search

In this cookbook, we'll demonstrate a flight search agent through LangGraph and Exfunc.

## Step 1: Set up the environment

First, let's install the required libraries and set up the SDK and clients.

In [ ]:
%pip install -U langgraph langchain langchain-openai

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key:\n")
if "EXFUNC_API_KEY" not in os.environ:
    os.environ["EXFUNC_API_KEY"] = getpass.getpass("Exfunc API key:\n")

In [ ]:
from langchain_openai import ChatOpenAI
from exfunc import Exfunc

model = ChatOpenAI(model="gpt-4o-mini")
s = Exfunc(api_key=os.getenv("EXFUNC_API_KEY"))

## Step 2: Add flight search endpoints as tools

Next, we will add custom tools of flight search endpoints to the agent.

In [ ]:
from langchain_core.tools import tool
from typing import Annotated

@tool
def search_one_way_flights(
    origin: Annotated[str, "Origin location"],
    destination: Annotated[str, "Destination location"],
    depart_date: Annotated[str, "The date of the departure. The format should be YYYY-MM-DD"],
) -> list:
    """Search one-way flights on Skyscanner"""
    search_one_way_result = s.skyscanner.search_one_way(request={
        "origin": origin,
        "destination": destination,
        "depart_date": depart_date,
    })
    return search_one_way_result.itineraries


@tool
def search_roundtrip_flights(
    origin: Annotated[str, "Origin location"],
    destination: Annotated[str, "Destination location"],
    depart_date: Annotated[str, "The date of the departure. The format should be YYYY-MM-DD"],
    return_date: Annotated[str, "The date of the return. The format should be YYYY-MM-DD"],
) -> list:
    """Search roundtrip flights on Skyscanner"""
    search_roundtrip_result = s.skyscanner.search_roundtrip(request={
        "origin": origin,
        "destination": destination,
        "depart_date": depart_date,
        "return_date": return_date,
    })
    return search_roundtrip_result.itineraries


tools = [search_one_way_flights, search_roundtrip_flights]

query = "Search roundtrip flights from san francisco to new york from december 12th to december 19th of 2024"

## Step 3: Create a pre-built ReAct agent with the tools

Using prebuilt agent from LangGraph, you can easily add tools to the agent and invoke with user query.

In [ ]:
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(model, tools)

messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

## Conclusion

That's it! By following this cookbook, you can easily create a flight search agent with LangGraph and Exfunc.

If you have any questions or need help, feel free to reach out to us at [Exfunc](https://exfunc.com/).